## 1. youtube api 기본 세팅

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

DEVEOPER_KEY="youtube api 키"
YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

In [854]:
youtube=build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVEOPER_KEY)

# 2. data set Load

In [1]:
import pandas as pd
import numpy as np

In [4]:
# watched_df=pd.read_csv('시청 기록 파일\[은호]시청 기록[new].csv')
watched_df=pd.read_csv('시청 기록 파일\[정흠]시청 기록[new].csv')
# watched_df=pd.read_csv('시청 기록 파일\[지애]시청 기록[new].csv')

# 60초 영상들이 sec가 1로 크롤링됨

In [5]:
watched_df[watched_df['title']=="한강 경찰이 음주 배 운전 잡는방법 #한강 #디즈니플러스"]

,title,id,date,category_id,view,like,comment,hour,min,sec,...,description,thumbnail,channel_title,tags,dimension,definition,caption,licensed_content,topic_ids,relevant_topic_ids
2279,한강 경찰이 음주 배 운전 잡는방법 #한강 #디즈니플러스,QwvygrDQ6RM,2023-09-14,1,4948467,62197,560,0,0,1,...,#SHORTS #권상우 #김희원 #성동일 #이상이 #배다빈 #신현승,https://i.ytimg.com/vi/QwvygrDQ6RM/default.jpg,펀메이커 - 영화 드라마 명장면,NaN,2d,hd,false,False,NaN,NaN


In [858]:
# '-' 값을 포함하는 행을 삭제
cols_to_check = ['category_id', 'view', 'like', 'comment', 'hour', 'min', 'sec','watched_time']
for col in cols_to_check:
    watched_df = watched_df[watched_df[col] != '-']

In [859]:
# 데이터 타입 변환
dtype_dict = {
    'category_id': 'int64',
    'view': 'int64',
    'like': 'int64',
    'comment': 'int64',
    'hour': 'int64',
    'min': 'int64',
    'sec': 'int64',
    'caption': 'bool',
    'licensed_content': 'bool',
    'topic_ids': 'float64',
    'relevant_topic_ids': 'float64'
}

for col, dtype in dtype_dict.items():
    watched_df[col] = watched_df[col].astype(dtype)


In [860]:
watched_df.dtypes

title                  object
id                     object
date                   object
category_id             int64
view                    int64
like                    int64
comment                 int64
hour                    int64
min                     int64
sec                     int64
watched_time           object
description            object
thumbnail              object
channel_title          object
tags                   object
dimension              object
definition             object
caption                  bool
licensed_content         bool
topic_ids             float64
relevant_topic_ids    float64
dtype: object

# sec가 1인영상들은 60초로 변환

In [861]:
watched_df.loc[watched_df['sec'] == 1, 'sec'] = 60

In [862]:
a=watched_df[['title', 'hour','min','sec']]

In [863]:
# a[((a['hour']==0) & a['min'] == 1) & (a['sec'] <= 1)]
a[a['sec']==1]

,title,hour,min,sec


# 3. 결측치 제거

In [864]:
watched_df.isnull().sum()

title                     0
id                        0
date                      0
category_id               0
view                      0
like                      0
comment                   0
hour                      0
min                       0
sec                       0
watched_time              0
description            7256
thumbnail                 0
channel_title             0
tags                  13296
dimension                 0
definition                0
caption                   0
licensed_content          0
topic_ids             31806
relevant_topic_ids    31806
dtype: int64

In [865]:
# 결측치가 너무 많은 'topic_ids', 'relevant_topic_ids'는 drop
watched_df = watched_df.drop(columns=['topic_ids', 'relevant_topic_ids']) #'tags','description'

In [866]:
watched_df.isnull().sum()

title                   0
id                      0
date                    0
category_id             0
view                    0
like                    0
comment                 0
hour                    0
min                     0
sec                     0
watched_time            0
description          7256
thumbnail               0
channel_title           0
tags                13296
dimension               0
definition              0
caption                 0
licensed_content        0
dtype: int64

In [867]:
print(len(watched_df))

31806


# 4. 카테고리 이름, timestamp 변환

* ### 카테고리 id를 카테고리 이름으로 변환

In [868]:
categories = {'1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music', '15': 'Pets & Animals', '17': 'Sports', '18': 'Short Movies', '19': 'Travel & Events', '20': 'Gaming', '21': 'Videoblogging', '22': 'People & Blogs', '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', '28': 'Science & Technology', '30': 'Movies', '31': 'Anime/Animation', '32': 'Action/Adventure', '33': 'Classics', '34': 'Comedy', '35': 'Documentary', '36': 'Drama', '37': 'Family', '38': 'Foreign', '39': 'Horror', '40': 'Sci-Fi/Fantasy', '41': 'Thriller', '42': 'Shorts', '43': 'Shows', '44': 'Trailers'}

In [869]:
watched_df['category_id'] = watched_df['category_id'].astype(str)
# 카테고리 id를 이름으로 변환
watched_df['category_id'] = watched_df['category_id'].map(categories)

In [870]:
watched_df['category_id']

0           Entertainment
1        Film & Animation
2                  Sports
3          People & Blogs
4                  Sports
               ...       
32731              Gaming
32732      People & Blogs
32733     News & Politics
32734      People & Blogs
32735      People & Blogs
Name: category_id, Length: 31806, dtype: object

* ### timestamp 변환

In [871]:
watched_df['watched_time']

0        2023-10-09T06:48:44.222Z
1        2023-10-09T06:45:56.257Z
2        2023-10-09T06:45:55.717Z
3        2023-10-09T06:45:55.274Z
4        2023-10-09T06:45:53.689Z
                   ...           
32731    2023-04-19T13:24:44.325Z
32732    2023-04-19T13:18:08.840Z
32733    2023-04-19T13:16:06.338Z
32734    2023-04-19T12:44:38.826Z
32735    2023-04-19T12:41:30.263Z
Name: watched_time, Length: 31806, dtype: object

In [872]:
from datetime import datetime

def format_time(iso_time):
    if iso_time is None:
        return None

    # 문자열 파싱
    dt = datetime.fromisoformat(iso_time.replace("Z", "+00:00"))
    # 형식 변환 (초까지 포함)
    return dt.strftime("%Y년 %m월 %d일 %H시 %M분 %S초")

# detail_df의 watched_time 열의 각 값에 대해 변환 적용
watched_df['watched_time'] = watched_df['watched_time'].apply(format_time)

In [873]:
watched_df[['watched_time', 'hour','min','sec']].head()

,watched_time,hour,min,sec
0,2023년 10월 09일 06시 48분 44초,0,0,25
1,2023년 10월 09일 06시 45분 56초,0,0,28
2,2023년 10월 09일 06시 45분 55초,0,0,57
3,2023년 10월 09일 06시 45분 55초,0,0,51
4,2023년 10월 09일 06시 45분 53초,0,0,59


# 5. Shorts 영상만 추출

In [874]:
# 60초 이내 영상을 shorts영상으로 분류하므로, 시간, 분이 0인 영상만 쇼츠영상으로 분류
Youtube_shorts = watched_df[(watched_df['min'] == 0) & (watched_df['hour'] == 0)]

In [875]:
Youtube_shorts.reset_index(drop=True, inplace=True)

이전 영상과 현재 영상의 시청 시각 차이로 현재 영상 시청 시간을 구함

In [877]:
import pandas as pd

# datetime으로 변환해서 계산
Youtube_shorts['watched_time'] = pd.to_datetime(Youtube_shorts['watched_time'], format="%Y년 %m월 %d일 %H시 %M분 %S초")

# 실제 시청 시간 계산 (이전 영상 시청 시각과의 차이)
Youtube_shorts['watched_duration'] = abs(Youtube_shorts['watched_time'].diff(1).dt.total_seconds().fillna(0))

C:\Users\USER\AppData\Local\Temp\ipykernel_8520\953162027.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Youtube_shorts['watched_time'] = pd.to_datetime(Youtube_shorts['watched_time'], format="%Y년 %m월 %d일 %H시 %M분 %S초")
C:\Users\USER\AppData\Local\Temp\ipykernel_8520\953162027.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Youtube_shorts['watched_duration'] = abs(Youtube_shorts['watched_time'].diff(1).dt.total_seconds().fillna(0))


In [878]:
Youtube_shorts[['watched_time', 'watched_duration']]

,watched_time,watched_duration
0,2023-10-09 06:48:44,0.0
1,2023-10-09 06:45:56,168.0
2,2023-10-09 06:45:55,1.0
3,2023-10-09 06:45:55,0.0
4,2023-10-09 06:45:53,2.0
...,...,...
25343,2023-04-19 14:01:39,12.0
25344,2023-04-19 14:01:37,2.0
25345,2023-04-19 14:00:36,61.0
25346,2023-04-19 14:00:06,30.0


In [879]:
print(Youtube_shorts['watched_duration'].describe())
print(Youtube_shorts['watched_duration'].info())

count    25348.000000
mean       588.659855
std       3629.192130
min          0.000000
25%          6.000000
50%         24.000000
75%         54.000000
max      74627.000000
Name: watched_duration, dtype: float64
<class 'pandas.core.series.Series'>
RangeIndex: 25348 entries, 0 to 25347
Series name: watched_duration
Non-Null Count  Dtype  
--------------  -----  
25348 non-null  float64
dtypes: float64(1)
memory usage: 198.2 KB
None


In [880]:
Youtube_shorts.reset_index(drop=True, inplace=True)

In [881]:
print("최종 shorts영상 개수:", len(Youtube_shorts))

최종 shorts영상 개수: 25348


In [882]:
Youtube_shorts[['watched_duration','sec']]

,watched_duration,sec
0,0.0,25
1,168.0,28
2,1.0,57
3,0.0,51
4,2.0,59
...,...,...
25343,12.0,58
25344,2.0,23
25345,61.0,60
25346,30.0,11


# 4. 사용하지 못하는 데이터 삭제

In [890]:
print("댓글수, 좋아요수, 조회수 없는 영상", len(Youtube_shorts)-len(Youtube_shorts[(Youtube_shorts['view'] != 0) & (Youtube_shorts['like'] != 0) & (Youtube_shorts['comment'] != 0)]))

댓글수, 좋아요수, 조회수 없는 영상 1696


In [891]:
# 댓글수, 좋아요수, 조회수를 막아놓은 영상은 제거
Youtube_shorts = Youtube_shorts[(Youtube_shorts['view'] != 0) & (Youtube_shorts['like'] != 0) & (Youtube_shorts['comment'] != 0)]

In [892]:
print("댓글수,좋아요수,조회수 없는 영상 제거 후 : ",len(Youtube_shorts))

댓글수,좋아요수,조회수 없는 영상 제거 후 :  23652


In [893]:
print("시청시간 60초 이상 영상 개수:",len(Youtube_shorts[Youtube_shorts['watched_duration']>60]))

시청시간 60초 이상 영상 개수: 4767


# IQR 기준으로 데이터 필터링

유튜브를 떠나서 시청 시간이 증가한 영상들을 어느정도 추정해서 제거하기위함

In [894]:
print("제거 전 맨처음 데이터 개수 : ", len(Youtube_shorts))

test=Youtube_shorts['watched_duration']

import numpy as np

def outlier_remove(data, threshold=1.5):
    q1, q3 = np.percentile(data, [25, 75])  # 1사분위수, 3사분위수 계산
    IQR = q3 - q1  # IQR 계산
    
    lower_bound = q1 - (threshold * IQR)  # Outlier 판단 Lower Bound 계산
    upper_bound = q3 + (threshold * IQR)  # Outlier 판단 Upper Bound 계산

    filtered_data = [x for x in data if x >= lower_bound and x <= upper_bound]
    outliers = [x for x in data if x < lower_bound or x > upper_bound]  # Fix variable name here
    return filtered_data, outliers, q1, q3, IQR, lower_bound, upper_bound

if __name__ == '__main__':
    X = test
    filtered_data, outliers, q1, q3, IQR, lower_bound, upper_bound = outlier_remove(X)
    
    
print("Filtered Data:", filtered_data)
print("Outliers:", outliers)
print("Q1:", q1)
print("Q3:", q3)
print("IQR:", IQR)
print("Lower Bound:", lower_bound)
print("Upper Bound:", upper_bound)
print("이상치 삭제된 개수: ", len(outliers))

# 데이터셋에서 이상치 제거
Youtube_shorts=Youtube_shorts[Youtube_shorts['watched_duration']<=124]

제거 전 맨처음 데이터 개수 :  23652
Filtered Data: [0.0, 1.0, 0.0, 2.0, 0.0, 1.0, 1.0, 1.0, 1.0, 28.0, 10.0, 34.0, 37.0, 12.0, 4.0, 74.0, 56.0, 50.0, 9.0, 20.0, 6.0, 11.0, 47.0, 6.0, 5.0, 26.0, 2.0, 6.0, 10.0, 6.0, 1.0, 68.0, 22.0, 3.0, 6.0, 19.0, 16.0, 48.0, 74.0, 101.0, 3.0, 19.0, 9.0, 46.0, 76.0, 64.0, 58.0, 1.0, 13.0, 44.0, 3.0, 47.0, 60.0, 48.0, 69.0, 9.0, 3.0, 55.0, 3.0, 14.0, 65.0, 77.0, 1.0, 52.0, 20.0, 1.0, 67.0, 67.0, 52.0, 5.0, 1.0, 60.0, 67.0, 4.0, 32.0, 59.0, 49.0, 12.0, 5.0, 4.0, 6.0, 1.0, 5.0, 25.0, 78.0, 18.0, 3.0, 31.0, 52.0, 7.0, 53.0, 13.0, 71.0, 44.0, 6.0, 5.0, 66.0, 1.0, 27.0, 10.0, 70.0, 1.0, 6.0, 2.0, 3.0, 19.0, 27.0, 48.0, 17.0, 4.0, 67.0, 41.0, 7.0, 64.0, 61.0, 21.0, 85.0, 33.0, 42.0, 7.0, 22.0, 2.0, 52.0, 53.0, 49.0, 54.0, 44.0, 5.0, 105.0, 11.0, 9.0, 1.0, 3.0, 0.0, 6.0, 1.0, 1.0, 1.0, 13.0, 52.0, 1.0, 4.0, 1.0, 4.0, 3.0, 5.0, 2.0, 1.0, 16.0, 3.0, 32.0, 15.0, 3.0, 2.0, 31.0, 60.0, 36.0, 1.0, 7.0, 1.0, 36.0, 19.0, 34.0, 10.0, 1.0, 40.0, 3.0, 34.0, 52.0, 37.0, 4.0, 3.0, 2.

In [895]:
print("최종 데이터 개수: ", len(Youtube_shorts))

최종 데이터 개수:  21443


# 5. TARGET값 라벨링

* 시청시간이 2초 이내인 영상들을 비선호 영상이라고 가정하고, 1의 값을 부여함

* 선호하는 영상은 0

In [896]:
Youtube_shorts['video_pref']=0
# 'watched_duration' 값이 2 이하인 행들에 대해 'video_pref' 값을 1로 설정
Youtube_shorts['video_pref'] = np.where(Youtube_shorts['watched_duration'] <= 2, 1, Youtube_shorts['video_pref'])    # Youtube_shorts['sec']//2
# print(len(Youtube_shorts[Youtube_shorts['video_pref']==1]))
# feature_extraction에서 사용할 변수 저장
Youtube_shorts.to_pickle(f"[정흠]Youtube_shorts.pkl")#, index=False, encoding='utf-8-sig')